# Práctico 1 - Parte 3 de 3

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

En esta tercera parte se arman los datasets, los dataloaders y se entrena y prueba el modelo.

## ⚠ IMPORTANTE ⚠

Por favor leer el archivo [Practico_1.md](https://github.com/grobiglio/deepleaning/blob/master/practico/Practico_1.md#deep-learning---trabajo-pr%C3%A1ctico-1) que se encuentra en el repositorio donde se puso este trabajo práctico.

## Importaciones

In [1]:
import mlflow
import tempfile
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from gensim import corpora
from tqdm.notebook import tqdm
from sklearn.metrics import balanced_accuracy_score
from practico1_modulo import *

## Constantes

In [2]:
EPOCHS = 1
BATCH_SIZE = 100

## Carga de datos

Carga de datos de entrenamiento

In [3]:
X_train = torch.load('./data/X_train.pt')
y_train = torch.load('./data/y_train.pt')

In [4]:
# La reducción del dataset de entrenamiento es temporal
# Cuando compruebe que funciona se eliminará esta celda.
X_train = X_train[:1000000]
X_train.shape

torch.Size([1000000, 17])

In [5]:
# La reducción del dataset de entrenamiento es temporal
# Cuando compruebe que funciona se eliminará esta celda.
y_train = y_train[:1000000]
y_train.shape

torch.Size([1000000])

Carga de datos de prueba

In [6]:
X_test = torch.load('./data/X_test.pt')
y_test = torch.load('./data/y_test.pt')

In [7]:
# La reducción del dataset de prueba es temporal.
# Cuando compruebe que funciona se eliminará esta celda.
X_test = X_test[:500000]
X_test.shape

torch.Size([500000, 16])

In [8]:
# La reducción del dataset de prueba es temporal.
# Cuando compruebe que funciona se eliminará esta celda.
y_test = y_test[:500000]
y_test.shape

torch.Size([500000])

## Embedding de títulos

In [9]:
# https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding
embeddings_matrix = torch.load('./data/embeddings_matrix.pt')
embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                          padding_idx=0)

## Construcción del Dataset

In [10]:
train_dataset = MeLiChallengeDataset(X_train, y_train)
test_dataset = MeLiChallengeDataset(X_test, y_test)

train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          drop_last=False)
i = 0
for data in tqdm(train_loader):
    i += 1
print(f'Recorrida exitosa de {i} batches de entrenamiento.')

test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         drop_last=False)
i = 0
for data in tqdm(test_loader):
    i += 1
print(f'Recorrida exitosa de {i} batches de prueba.')

  0%|          | 0/10000 [00:00<?, ?it/s]

Recorrida exitosa de 10000 batches de entrenamiento.


  0%|          | 0/5000 [00:00<?, ?it/s]

Recorrida exitosa de 5000 batches de prueba.


## Construcción del Modelo

In [11]:
modelo = MeLiChallengeClassifier(embeddings)

## Algoritmo de Optimización

In [12]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.001, momentum=0.9)

## Entrenamiento del modelo

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Utilizando {device}')
modelo.to(device)

Utilizando cuda


MeLiChallengeClassifier(
  (embeddings): Embedding(50002, 300, padding_idx=0)
  (hidden1): Linear(in_features=300, out_features=300, bias=True)
  (hidden2): Linear(in_features=300, out_features=500, bias=True)
  (output): Linear(in_features=500, out_features=632, bias=True)
)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    '''Entrenamiento de una red neuronal.
    
    Parámetros:
    -----------
    - dataloader: Iterador (objeto) de Pytorch construido en base al dataset basado en la clase MeLiChallengeDataset.
    - model: Modelo (objeto) basado en la clase MeLiChallengeClassifier.
    - loss_fn: Función de costo.
    - optimizer: Optimizador.
    
    Salidas:
    --------
    running_loss: Lista con los valores de la función de costo minimizados.
    
    '''
    size = len(dataloader.dataset)
    model.train()
    running_loss = []
    for batch, data in enumerate(dataloader):
        X, y = data['data'].to(device), data['target'].to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        # pred: Dimensión 100 x 632
        # y: Dimensión 100
        # Por eso esto no funciona 😡 ¿Cómo se soluciona? 😡
        loss.backward()
        optimizer.step()
        running_loss.append(loss.item())
        
        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    return running_loss

## Evaluación del Modelo

In [15]:
def test(dataloader, model, loss_fn):
    '''Evaluación de una red neuronal.
    
    Parámetros:
    -----------
    - dataloader: Iterador (objeto) de Pytorch construido en base al dataset basado en la clase MeLiChallengeDataset.
    - model: Modelo (objeto) basado en la clase MeLiChallengeClassifier.
    - loss_fn: Función de costo.
    
    Salidas:
    --------
    - running_loss: Lista con los valores de la función de costo minimizados.
    - targets: Lista con los valores verdaderos del objetivo.
    - predictions: Lista con los valores predichos.
    ⚠ ATENCIÓN ⚠ Aquí hay un problema porque los targets son valores enteros y
    las predicciones son decimales. No se pueden comparar para obtener un accuracy.
    '''
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    running_loss = []
    targets = []
    predictions = []
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data in dataloader:
            X, y = data['data'].to(device), data['target'].to(device)
            pred = model(X)
            y_true = y # Dimensión 100
            y_pred = pred # Dimensión 100 x 632
        try:
            accuracy = balanced_accuracy_score(y_true, y_pred)
            print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%")
        except:
            print('No es posible calcular el accuracy porque no coincide \
dimensión del valor predicho con la dimensión del valor verdadero.')
        running_loss.append(loss_fn(pred, y))
        targets.extend(y.cpu().detach().numpy())
        predictions.extend(pred.cpu().squeeze().detach().numpy())
                            
    return running_loss, targets, predictions

In [16]:
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    _ = train(train_loader, modelo, loss_function, optimizer)
    _, _, _ = test(test_loader, modelo, loss_function)
print("!Listo!")

Epoch 1
-------------------------------
loss: 6.451149  [    0/1000000]
loss: 6.448824  [100000/1000000]
loss: 6.448419  [200000/1000000]
loss: 6.447014  [300000/1000000]
loss: 6.446904  [400000/1000000]
loss: 6.446173  [500000/1000000]
loss: 6.446985  [600000/1000000]
loss: 6.444613  [700000/1000000]
loss: 6.445238  [800000/1000000]
loss: 6.446004  [900000/1000000]
No es posible calcular el accuracy porque no coincide dimensión del valor predicho con la dimensión del valor verdadero.
!Listo!


## ¿Por qué sospechamos que no funciona?

La red neuronal tiene como salida un vector de 632 componentes el cual es comparado con un solo componente, la etiqueta asocada a la clasificación del título. Entiendemos que allí hay una incompatibilidad que el código acepta (no da error) pero que evidentemente no permite que la red aprenda.

## Optimización de hiperparámetros

In [ ]:
mlflow.set_experiment("practico_1")

with mlflow.start_run():
    mlflow.log_param("model_name", "mlp")
    mlflow.log_params({
        "embedding_size": 300,
        "hidden1_size": 128,
        "hidden2_size": 128
    })
    modelo = MeLiChallengeClassifier(embeddings)
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(modelo.parameters(), lr=0.001, weight_decay=1e-5)
    modelo.to(device)
    for epoch in range(EPOCHS):
        print(f"Epoch {t+1}\n-------------------------------")
        # Entrenamiento del modelo
        running_loss = train(train_loader, modelo, loss_function, optimizer)
        '''
        modelo.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            output = modelo(batch["data"])
            loss_value = loss(output, batch["target"])
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())
        '''
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        # Evaluación del modelo
        running_loss, targets, predictions = test(test_loader, modelo, loss_function)
        '''
        modelo.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = modelo(batch["data"])
            running_loss.append(loss(output, batch["target"]).item())
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().detach().numpy())
        '''
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
        try:
            mlflow.log_metric("test_avp", balanced_accuracy_score(targets, predictions), epoch)
        except:
            targets = [1, 0, 0] # Valor dummie para que no de error
            predictions = [0, 1, 0] # Valor dummie para que no de error
            # El problema es que targets son valores enteros y predictions decimales
            mlflow.log_metric("test_avp", balanced_accuracy_score(targets, predictions), epoch)
    
    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            X = batch['data'].to(device)
            output = modelo(X)
            targets.extend(batch["target"].numpy())
            predictions.extend(output.cpu().squeeze().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(f"{tmpdirname}/predictions.csv.gz", index=False)
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

2022/11/08 23:06:32 INFO mlflow.tracking.fluent: Experiment with name 'practico_1' does not exist. Creating a new experiment.


Epoch 1
-------------------------------
loss: 6.448898  [    0/1000000]
loss: 6.447157  [100000/1000000]
loss: 6.448918  [200000/1000000]
loss: 6.447146  [300000/1000000]
loss: 6.448543  [400000/1000000]
loss: 6.449569  [500000/1000000]
loss: 6.448979  [600000/1000000]
loss: 6.448914  [700000/1000000]
loss: 6.447817  [800000/1000000]
loss: 6.449285  [900000/1000000]
No es posible calcular el accuracy porque no coincide dimensión del valor predicho con la dimensión del valor verdadero.


  0%|          | 0/5000 [00:00<?, ?it/s]